In [5]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown
from typing import List

In [2]:
load_dotenv(override=True)

True

In [3]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [30]:
number_of_searches = 3

class WebSearchItem(BaseModel):
    reason: str = Field(description="The reason for the search")
    query: str = Field(description="The query to search for")

class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="The list of search items")


In [31]:
planner_instructions = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {number_of_searches} terms to query for."


In [32]:
planner_agent = Agent(name="plannerAgent", instructions=planner_instructions,model="gpt-4o-mini", output_type=WebSearchPlan)

In [33]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("sukhdeepnarulasingh@gmail.com") 
    to_email = To("sukhdeepsingh1612@gmail.com") 
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [34]:
email_instructions = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="emailer agent",
    instructions=email_instructions,
    tools=[send_email],
    model="gpt-4o-mini"
    
)

In [12]:
writer_instructions = ( "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)

class Report_Data(BaseModel):
    short_summary: str = Field(description="A short summary of the report")

    markdown_report: str = Field(description="The report in markdown format")

    follow_up_questions: List[str] = Field(description="A list of follow-up questions that the reader may have")

writer_agent = Agent(
    name="Writer agent",
    instructions=writer_instructions,
    model="gpt-4o-mini",
    output_type=Report_Data,
)

In [37]:
async def plan_searches(query: str):
    print(f"Planning searches for query: {query}")
    result = await Runner.run(planner_agent, f"Query :{query}")
    return result.final_output

async def perform_searches(search_plan : WebSearchPlan):
    print(f"Performing searches for query: {search_plan}")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searches")
    return results

async def search(item: WebSearchItem):
    print(f"Searching for query: {item.query}")
    input = f"Search term : {item.query}\n Reason for search: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

In [38]:
async def write_report(query: str, search_results: List[str]):
    print(f"Writing report for query: {query}")
    input = f"Query: {query}\nSearch results: {search_results}"
    resullt = await Runner.run(writer_agent, input)
    return resullt.final_output

async def send_email(report: Report_Data):
    print(f"Sending email for report: {report}")
    input = f"Report: {report.markdown_report}"
    result = await Runner.run(email_agent, input)
    print("Email sent")


In [39]:
query = "Best car to buy under 10 lakhs in India"

search_plan = await plan_searches(query)
search_results = await perform_searches(search_plan)
report = await write_report(query, search_results)
await send_email(report)
print("----Done----")

Planning searches for query: Best car to buy under 10 lakhs in India
Performing searches for query: searches=[WebSearchItem(reason='To find overall best cars in India under 10 lakhs based on reviews and expert opinions.', query='best car under 10 lakhs India 2023 review'), WebSearchItem(reason='To gather consumer feedback and popular choices for cars within the budget.', query='most popular cars under 10 lakhs India 2023'), WebSearchItem(reason='To check for recent models and their specifications along with pricing.', query='latest cars under 10 lakhs India 2023')]
Searching for query: best car under 10 lakhs India 2023 review
Searching for query: most popular cars under 10 lakhs India 2023
Searching for query: latest cars under 10 lakhs India 2023
Finished searches
Writing report for query: Best car to buy under 10 lakhs in India
Sending email for report: short_summary='This report evaluates the best cars to buy in India under ₹10 lakh in 2023, providing details on performance, featur